In [11]:
import os
import fastavro
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from correction.compute import *

In [203]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [224]:
data = pd.read_csv("../data_examples/csv/raw_detections.csv", index_col=False)

In [227]:
# test c2
data.at[180, "isdiffpos"] = 'f'

In [234]:
# test c3
data.at[150, "distnr"] = 0.1

In [181]:
print(data.objectId.unique(), data.fid.unique())

['ZTF18aazxcwf' 'ZTF20aaelulu' 'ZTF19aazzpje' 'ZTF18abdgukn'
 'ZTF19abaejrh' 'ZTF18aaiscil'] [1 2]


In [235]:
pd.options.display.max_rows = 999
display(data[["objectId", "fid", "magpsf", "distnr", "isdiffpos"]])

,objectId,fid,magpsf,distnr,isdiffpos
0,ZTF18aazxcwf,1,16.970957,0.118913,f
1,ZTF18aazxcwf,1,17.193100,0.087401,f
2,ZTF18aazxcwf,2,16.954599,0.086154,f
3,ZTF18aazxcwf,2,18.716200,1.045030,t
4,ZTF18aazxcwf,2,17.118080,0.046982,f
5,ZTF18aazxcwf,1,17.481110,0.698191,f
6,ZTF18aazxcwf,2,16.838499,0.409430,f
7,ZTF18aazxcwf,1,17.014387,0.101945,f
8,ZTF18aazxcwf,1,17.172487,0.201816,f
9,ZTF18aazxcwf,1,17.012300,0.102877,f


In [236]:
def apply_correction_df(data):
    
    # apply correction where possible, save corrected column
    df = data.copy()
    #display(df.head(1))
    #print(df.objectId.unique(), df.fid.unique())
    df['isdiffpos'] = df['isdiffpos'].map({'t': 1., 'f': -1.})
    df["corr"] = df["distnr"] < 1.4
    correction_results = df.apply(lambda x: correction(x.magnr, x.magpsf, x.sigmagnr, x.sigmapsf, x.isdiffpos) if x["corr"] else (np.nan, np.nan, np.nan), axis=1, result_type="expand")
    df["magpsf_corr"], df["sigmapsf_corr"], df["sigmapsf_corr_ext"] = correction_results[0], correction_results[1], correction_results[2]
    
    # check for consistency
    idxmin = df.candid.idxmin()
    df["corr_magstats"] = df.loc[idxmin]["corr"]
    
    # check if suspicious behavior in the light curve (c1 | c2 | c3)
    mask = ((df["corr"] == False) & (df.isdiffpos == -1)) | (df.corr_magstats & (df["corr"] == False)) | ((df.corr_magstats == False) & df["corr"])
    df["dubious"] = mask
    
    return df

In [237]:
dflarge = data.groupby(["objectId", "fid"]).apply(apply_correction_df)

In [243]:
dflarge.loc[dflarge.dubious][["objectId", "fid", "distnr", "corr", "corr_magstats", "dubious"]]

,,,objectId,fid,distnr,corr,corr_magstats,dubious
objectId,fid,,,,,,,
ZTF18aaiscil,1,315,ZTF18aaiscil,1,1.621020,False,True,True
ZTF19aazzpje,2,180,ZTF19aazzpje,2,2.503458,False,False,True
ZTF20aaelulu,2,150,ZTF20aaelulu,2,0.100000,True,False,True


In [34]:
a.loc[a['corr'] == False].head()

,objectId,fid,candid,jd,pid,diffmaglim,pdiffimfilename,programpi,programid,isdiffpos,...,maggaia,maggaiabright,exptime,drb,drbversion,candid_parent,corr,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ext
113,ZTF20aaelulu,2,1101542291015015004,2.458856e+06,1101542291015,20.367029,ztf_20200107542268_000576_zr_c03_o_q3_scimrefd...,Kulkarni,1,1.0,...,17.880779,-999.0,30.0,1.000000,d6_m7,NaN,False,NaN,NaN,NaN
114,ZTF20aaelulu,2,1105481241015015001,2.458860e+06,1105481241015,19.761808,ztf_20200111481238_000576_zr_c03_o_q3_scimrefd...,Kulkarni,1,1.0,...,17.880779,-999.0,30.0,0.999998,d6_m7,NaN,False,NaN,NaN,NaN
117,ZTF20aaelulu,2,1113481561015015006,2.458868e+06,1113481561015,20.348034,ztf_20200119481562_000576_zr_c03_o_q3_scimrefd...,Kulkarni,1,1.0,...,17.880779,-999.0,30.0,0.999998,d6_m7,NaN,False,NaN,NaN,NaN
119,ZTF20aaelulu,2,1117502231015015005,2.458872e+06,1117502231015,20.242445,ztf_20200123502234_000576_zr_c03_o_q3_scimrefd...,Kulkarni,1,1.0,...,17.880779,-999.0,30.0,0.999987,d6_m7,NaN,False,NaN,NaN,NaN
121,ZTF20aaelulu,2,1121521071015015006,2.458876e+06,1121521071015,20.243944,ztf_20200127521065_000576_zr_c03_o_q3_scimrefd...,Kulkarni,1,1.0,...,17.880779,-999.0,30.0,0.999996,d6_m7,NaN,False,NaN,NaN,NaN


In [163]:
grupos = data.groupby("objectId")

In [168]:
for g in grupos:
    re = apply_correction_df(g[1])